# Private GitHub Setup
1. Create a personal access token (classic) with the **repo** scope at https://github.com/settings/tokens.
2. Keep the token secret—do not share it or commit it to source control.
3. When prompted below, enter your GitHub username and paste the token; the notebook stores it only for this session.
4. If you prefer, set the variables ahead of time with `%env GITHUB_USERNAME=<user>` and `%env GITHUB_TOKEN=<token>`.
5. Similarly you can `%env PROJECT_NAME=<your github project>`

In [ ]:

import getpass
import os

if "GITHUB_USERNAME" not in os.environ or not os.environ["GITHUB_USERNAME"].strip():
    os.environ["GITHUB_USERNAME"] = input("Enter your GitHub username: ").strip()

if "GITHUB_TOKEN" not in os.environ or not os.environ["GITHUB_TOKEN"].strip():
    os.environ["GITHUB_TOKEN"] = getpass.getpass("Enter your GitHub personal access token: ").strip()

if not os.environ["GITHUB_TOKEN"]:
    raise RuntimeError("GITHUB_TOKEN is required to clone the private repository.")

In [ ]:
import os
import stat
from pathlib import Path

# Store credentials in ~/.netrc so git can authenticate without printing the token.
netrc_path = Path.home() / ".netrc"
netrc_content = (
    "machine github.com\n"
    f"login {os.environ['GITHUB_USERNAME']}\n"
    f"password {os.environ['GITHUB_TOKEN']}\n"
)
netrc_path.write_text(netrc_content)
netrc_path.chmod(stat.S_IRUSR | stat.S_IWUSR)
print(f"Credentials stored in {netrc_path}")

In [ ]:
import subprocess
from pathlib import Path
if "PROJECT_NAME" not in os.environ or not os.environ["PROJECT_NAME"].strip():
    os.environ["PROJECT_NAME"] = input("Enter your project name: ").strip()

project_name = os.environ["PROJECT_NAME"]
repo_url = f"https://github.com/vmorozov/{project_name}.git"
target_dir = Path(project_name)

if not target_dir.exists():
    subprocess.run(["git", "clone", repo_url], check=True)
else:
    print("Repository already exists; updating it with git pull.")
    subprocess.run(["git", "-C", str(target_dir), "pull", "--ff-only"], check=True)

In [ ]:
import os
from pathlib import Path

project_root = Path(project_name).resolve()
os.chdir(project_root)
print(f"Working directory set to: {project_root}")

In [ ]:
import os
from pathlib import Path

import torch

requirements_path = Path("requrment.txt")
if not requirements_path.exists():
    raise FileNotFoundError("requrment.txt is missing; verify the file name inside the repository.")

torch_version = torch.__version__.split("+")[0]
cuda_version = torch.version.cuda
cuda_suffix = f"cu{cuda_version.replace('.', '')}" if cuda_version else "cpu"
index_url = f"https://data.pyg.org/whl/torch-{torch_version}+{cuda_suffix}.html"
print(f"Using PyG wheel index: {index_url}")
os.environ["PYG_WHEEL_INDEX"] = index_url

!pip install --upgrade pip
!pip install -r requrment.txt -f $PYG_WHEEL_INDEX

In [ ]:
import subprocess
from pathlib import Path

output_notebook = Path("eval_output.ipynb")
command = [
    "jupyter",
    "nbconvert",
    "--to",
    "notebook",
    "--execute",
    "eval.ipynb",
    "--output",
    str(output_notebook),
    "--ExecutePreprocessor.timeout=900"
]
subprocess.run(command, check=True)
print(f"Notebook execution complete. Output saved to {output_notebook.resolve()}")